In [ ]:
#This code is for TF-2 D-BLSTM network.


import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import LeaveOneGroupOut
from keras.utils import np_utils
import keras    
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import Masking
from keras.layers import Dropout


logo = LeaveOneGroupOut()  #create the object logo for doing the leave-oen-subject-out cross validation
grp = numpy.loadtxt("Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/speaker_group.csv", delimiter=",")
data=numpy.loadtxt('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/time_data_2_step_forecast.csv',delimiter=",")
Y=numpy.loadtxt('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/label_2_step_forecast.csv',delimiter=",")
# we have to reshape the data for using it in LSTM/BLSTM network
A=data.reshape((1763,50,895))
#code for BLSTM model
F2con={} #save the confusion matrix
F2ACC={} #save the accuracy or weighted recall
F2WR={}
F2UWR={} #save the unweighted recall
F=0

F2predict={} #save the predicted result
F2test={} # save the actual test GT labels
F2_modelpred={}  #save the predicted softmax labels
for train, test in logo.split(A, Y, grp):
    label_train=np_utils.to_categorical(Y[train])
    # Set callback functions to early stop training and save the best model so far
    callbacks= [EarlyStopping(monitor='val_loss', patience=10)]
    
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(50, 895)))  # the masking layer will ignore the masked o's
    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dropout(0.5))
    #model.add(Bidirectional(LSTM(128)))
    #model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))##
    #model.add(Dense(256,activation='relu'))
    #model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax'))
    adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)   
    sgd=keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    adagrad=keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    history=model.fit(A[train], label_train,  epochs=50, validation_split=0.33, callbacks=callbacks, batch_size=128, verbose=1)
    X_pred=model.predict(A[test,:])
    Y_pred=numpy.argmax(X_pred, axis=1)
    F2VAL_ACC[F]=history.history['val_acc']
    F2TR_ACC[F]=history.history['acc']
    F2con[F]=confusion_matrix(Y[test],Y_pred)
    F2ACC[F]=accuracy_score(Y[test],Y_pred)
    F2WR[F]=recall_score(Y[test],Y_pred, average='weighted') 
    F2UWR[F]=recall_score(Y[test],Y_pred, average='macro') 
    F2predict[F]=X_pred
    F2test[F]=Y[test]
    F2_modelpred[F]=Y_pred
    print(F2con[F])
    print(F2UWR[F])
    print('score of F is {0}'.format(F))
    model.save('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/Results/BLSTM/Saved_Models/M_{}/m_model_sp_{}.h5'.format(cs, F))
    F=F+1

In [ ]:

###LSTM
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
#import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneGroupOut
from keras.utils import np_utils
import keras    
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import Masking
from keras.layers import Dropout
# fix random seed for reproducibility
numpy.random.seed(7)


F2predict={}
F2test={}
F2_modelpred={}


logo = LeaveOneGroupOut()
grp = numpy.loadtxt("Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/speaker_group.csv", delimiter=",")
data=numpy.loadtxt('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/time_data_2_step_forecast.csv',delimiter=",")
Y=numpy.loadtxt('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/CSV/label_2_step_forecast.csv',delimiter=",")
A=data.reshape((1763,50,895))
#code for BLSTM model
F2con={}
F2ACC={}
F2WR={}
F2UWR={}
F=0

for train, test in logo.split(A, Y, grp):
    label_train=np_utils.to_categorical(Y[train])
    # Set callback functions to early stop training and save the best model so far
    callbacks= [EarlyStopping(monitor='val_loss', patience=10)]
    
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(50, 895)))
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    #model.add(Bidirectional(LSTM(128)))
    #model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))##
    #model.add(Dense(256,activation='relu'))
    #model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax'))
    adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)   
    sgd=keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    adagrad=keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


    history=model.fit(A[train], label_train,  epochs=50, validation_split=0.2, callbacks=callbacks, batch_size=128, verbose=1)
    X_pred=model.predict(A[test,:])
    Y_pred=numpy.argmax(X_pred, axis=1)
    F2VAL_ACC[F]=history.history['val_acc']
    F2TR_ACC[F]=history.history['acc']
    F2con[F]=confusion_matrix(Y[test],Y_pred)
    F2ACC[F]=accuracy_score(Y[test],Y_pred)
    F2WR[F]=recall_score(Y[test],Y_pred, average='weighted') 
    F2UWR[F]=recall_score(Y[test],Y_pred, average='macro') 
    F2predict[F]=X_pred
    F2test[F]=Y[test]
    F2_modelpred[F]=Y_pred
    print(F2con[F])
    print(F2UWR[F])
    print('score of F is {0}'.format(F))
    model.save('Sadat/IEMOCAP_forcasting/sameframe/STATISTICAL/2_group/Results/LSTM/Saved_Models/M_{}/m_model_sp_{}.h5'.format(cs, F))
    F=F+1